# Search Tool Greek Hotels: GoogleAPI

----
Libraries:

In [1]:
import re
import numpy as np
import pandas as pd
import json 
from batoomer.twitter_nodes.search_engine import GoogleSearchEngine
from googleapiclient.errors import HttpError
from tqdm.notebook import tqdm
import time

---
Load Google Credentials and initialize GoogleSearchEngine

###### keys = []
with open('../../../../google_credentials.json', 'r') as f:
    keys = json.load(f)

se = GoogleSearchEngine(google_api_key=keys['api_key'], search_engine_id=keys['twitter_seID'])

---
## Dataset

In [3]:
data = pd.read_csv('../../../../datasets/Hotels/search-tool/greek-hotels-2018-search.csv', delimiter=';')
data.replace(np.nan, '', inplace=True)
data.head(5)

,Hotel City,Hotelname,Twitter Handle,Website
0,Faliraki,Hotel Calypso Palace,,https://www.calypsopalace.gr/
1,Platanes,Nefeli Hotel,,https://nefeli-hotel.gr/
2,Kallithea,Hotel Kalithea Horizon Royal,,https://www.kalitheahorizon.gr/
3,Archangelos,Porto Angeli Beach Resort Hotel,portoangeli,https://www.portoangeli.com/
4,Faliraki,Hotel Calypso Beach,,http://www.calypsohotel.gr/


---
## Search: Hotel Names
First we search Using the Hotels name. Than we merge our results with the dataset.

In [4]:
queries = list(data['Hotelname'])
results = pd.DataFrame()

for query in tqdm(queries):
    try:
        se.search(query=query)
        result = se.get_results()
        results= results.append(result)
    except HttpError as err:
        if err.resp.status == 429:
            print('Api Limit Hit!')
            time.sleep(60 * 60 * 24)
            se.search(query=query)
            result = se.get_results()
            results= results.append(result)

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
results = (results
           .replace(np.nan, '')
           .rename(columns={'Query':'Hotelname'})
           .reset_index()
           .drop('index', axis=1))
results = pd.merge(data, results, on=results.index, how='outer').drop(['key_0', 'Hotelname_y'], axis =1).rename(columns={'Hotelname_x':'Hotelname'})
results.head(5)

,Hotel City,Hotelname,Twitter Handle,Website,Result 1,Result 2,Result 3,Result 4,Result 5,Result 6,Result 7,Result 8,Result 9,Result 10
0,Faliraki,Hotel Calypso Palace,,https://www.calypsopalace.gr/,@MMGYglobal,@VillaChalkidiki,@diamondpalaceho,@PlanetEarthRec,,,,,,
1,Platanes,Nefeli Hotel,,https://nefeli-hotel.gr/,@NEFELIHOMES,@RoyalBlueResort,@paphotels,@NefeliNikiforou,@petrinosuites,,,,,
2,Kallithea,Hotel Kalithea Horizon Royal,,https://www.kalitheahorizon.gr/,@sheratonrhodes,@CaesarsGardens,@dmarisbay,@apollobeachsa,,,,,,
3,Archangelos,Porto Angeli Beach Resort Hotel,portoangeli,https://www.portoangeli.com/,@krpec,@apollobeachsa,@AvraBeachResort,@sheratonrhodes,@dmarisbay,,,,,
4,Faliraki,Hotel Calypso Beach,,http://www.calypsohotel.gr/,@CalypsoTowerlll,@CalypsoQueenFL,@medplaya,@TimAtenKnows,,,,,,


In [7]:
results.to_csv('Google-Search-Hotelname-Hotels.csv', index=False)